# AFAP Cash Flow Engine — Validation Notebook

This notebook validates the **AFAP Phase 3 Cash Flow Health Engine**.

The purpose of this notebook is to:
1. Confirm deterministic execution of `cash_flow_engine`
2. Validate that outputs conform to AFAP Phase 3 schema requirements
3. Inspect representative engine outputs for interpretability
4. Ensure internal schema validation is enforced

This notebook is part of **AFAP Phase 3 — Locked Engines**.


## Environment Setup

This cell configures the Python environment so the AFAP project modules  
can be imported correctly when running the notebook from the `/notebooks` directory.

We explicitly append the project root to `sys.path` to enable absolute imports
from the `engines` package.


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))


## Import Cash Flow Engine

We import the **Cash Flow Health Engine**, which:
- Computes operating profit
- Derives a financing coverage proxy
- Applies deterministic severity rules
- Performs internal schema validation before returning results


In [2]:
from engines.cash_flow_engine import cash_flow_engine


## Load Financial Statement Data

This dataset contains cleaned, standardized financial statement data
used across all AFAP engines.

Expected columns include:
- Company
- Year
- FS Category
- FS Subcategory
- Amount

The Cash Flow Engine relies on this structure to compute operating performance
and financing pressure indicators.


In [3]:
import pandas as pd

financials = pd.read_csv("../data/cleaned/Kenya_Airways.csv")
financials.head()


,Company,Year,FS Category,FS Subcategory,Amount
0,Kenya Airways,2021,Assets,Current Assets,"25,685"
1,Kenya Airways,2021,Assets,Non-Current Assets,"129,870"
2,Kenya Airways,2021,Assets,Inventory,"2,152"
3,Kenya Airways,2021,Liabilities,Current Liabilities,"80,965"
4,Kenya Airways,2021,Liabilities,Non-Current Liabilities,"157,927"


## Execute Cash Flow Engine

We now run the Cash Flow Engine against the full financials dataset.

Internally, the engine:
- Groups data by Company and Year
- Computes operating profit
- Calculates a coverage proxy
- Assigns a top-level severity
- Generates a human-readable explanation
- Validates output structure via `validate_engine_output`


In [4]:
cash_flow_results = cash_flow_engine(financials)


✅ cash_flow_engine output validated successfully.


## Inspect Engine Output

We inspect the first few results to confirm:

- Engine metadata is present
- Metrics are correctly computed
- Flags are deterministic and boolean
- Severity is top-level (not embedded in flags)
- Explanation text aligns with severity logic

This inspection confirms readiness for orchestration.


In [5]:
cash_flow_results[:2]


[{'engine': 'cash_flow_engine',
  'Company': 'Kenya Airways',
  'Year': np.int64(2021),
  'metrics': {'operating_profit': np.int64(-83827),
   'coverage_proxy': np.float64(-8.930116118035581)},
  'flags': {'negative_operating_profit': np.True_, 'weak_coverage': np.True_},
  'severity': 'action',
  'explanation': 'Operating activities do not appear to generate sufficient cash to cover financing obligations.'},
 {'engine': 'cash_flow_engine',
  'Company': 'Kenya Airways',
  'Year': np.int64(2022),
  'metrics': {'operating_profit': np.int64(-128019),
   'coverage_proxy': np.float64(-3.891983096707506)},
  'flags': {'negative_operating_profit': np.True_, 'weak_coverage': np.True_},
  'severity': 'action',
  'explanation': 'Operating activities do not appear to generate sufficient cash to cover financing obligations.'}]

## Validation Notes

✔ Output schema validation is enforced **inside the engine**  
✔ Severity is emitted as a top-level field  
✔ Metrics are numeric or null-safe  
✔ Flags are deterministic and auditable  
✔ Explanations are non-prescriptive and consistent  

This confirms that `cash_flow_engine` meets **AFAP Phase 3 compliance** and
is safe for orchestration and downstream AI interpretation.
